# Keras model fitting

This notebook trains a simple neural net using Keras and assesses its performance.

In [ ]:
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

This notebook is parameterized to work with [Papermill](https://papermill.readthedocs.io).
The following cell contains the default values of the parameters.

In [ ]:
input_file = "test_dataset.npz"
log_dir = "logs"

n_epochs = 10
n_batch = 32
verbose = 1

First we load the training dataset.

In [ ]:
dset = np.load(input_file)
print(f"Number of training samples: {len(dset['X_train'])}.")
print(f"Number of test samples: {len(dset['X_test'])}.")

In [ ]:
n_classes = 10
y_train = tf.one_hot(dset["y_train"], n_classes)

X_train = dset["X_train"].astype(np.float32)
X_test = dset["X_test"].astype(np.float32)

Then define a MLP model, using Keras.

In [ ]:
n_features = dset["X_train"].shape[1]

model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=n_features),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(n_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.01),
    metrics=["accuracy"],
)

Fitting will take more or less time depending on the total number of epochs used.

In [ ]:
log_dir = log_dir + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
%%time
history = model.fit(
    X_train,
    y_train,
    epochs=n_epochs,
    batch_size=n_batch,
    verbose=verbose,
    callbacks=[tensorboard_callback],
)

Finally, we check the accuracy on the test dataset.

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
mlp_acc = accuracy_score(dset["y_test"], y_pred)
print(f"MLP test accuracy is {mlp_acc * 100:.2f}%.")

We can investigate the results via Tensorboard, use the integrated reverse proxy to access it.

In [ ]:
%env TENSORBOARD_PROXY_URL /user-redirect/proxy/%PORT%/

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs